<a href="https://colab.research.google.com/github/AjeetSingh02/spark_learn/blob/master/MLWithApacheSparkAndXGBoostOnGPU(package%20error).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Sat May  2 09:20:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
! wget https://repo1.maven.org/maven2/ai/rapids/cudf/0.9.2/cudf-0.9.2-cuda10-1.jar
! wget https://repo1.maven.org/maven2/ai/rapids/xgboost4j_2.x/1.0.0-Beta5/xgboost4j_2.x-1.0.0-Beta5.jar
! wget https://repo1.maven.org/maven2/ai/rapids/xgboost4j-spark_2.x/1.0.0-Beta5/xgboost4j-spark_2.x-1.0.0-Beta5.jar

--2020-05-02 09:21:24--  https://repo1.maven.org/maven2/ai/rapids/cudf/0.9.2/cudf-0.9.2-cuda10-1.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.64.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.64.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64759406 (62M) [application/java-archive]
Saving to: ‘cudf-0.9.2-cuda10-1.jar’

cudf-0.9.2-cuda10-1 100%[===================>]  61.76M  65.3MB/s    in 0.9s    

2020-05-02 09:21:25 (65.3 MB/s) - ‘cudf-0.9.2-cuda10-1.jar’ saved [64759406/64759406]

--2020-05-02 09:21:27--  https://repo1.maven.org/maven2/ai/rapids/xgboost4j_2.x/1.0.0-Beta5/xgboost4j_2.x-1.0.0-Beta5.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.200.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.200.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 349225621 (333M) [application/java-archive]
Saving to: ‘xgboost4j_2.x-1.0.0-Beta5.jar’

xgboost4j_2.x-1.0.0 100%[================

In [4]:
!ls
!pwd

cudf-0.9.2-cuda10-1.jar    spark-2.4.5-bin-hadoop2.7.tgz
sample_data		   xgboost4j_2.x-1.0.0-Beta5.jar
spark-2.4.5-bin-hadoop2.7  xgboost4j-spark_2.x-1.0.0-Beta5.jar
/content


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
os.environ["PYSPARK_SUBMIT_ARGS"] = '--jars /content/cudf-0.9.2-cuda10-1.jar,/content/xgboost4j_2.x-1.0.0-Beta5.jar,/content/xgboost4j-spark_2.x-1.0.0-Beta5.jar'

In [0]:
# This step was not in video. But since there was some error in cell below this was the solution of StackOverflow
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[2] pyspark-shell"

In [0]:
import findspark
findspark.init()   # Will search for Spark and set in the system path

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.executer.memory", "2g")
spark.conf.set("spark.driver.memory", "2g")

spark.sparkContext.addPyFile("/content/xgboost4j-spark_2.x-1.0.0-Beta5.jar")

In [0]:
import numpy as np
import pandas as pd
from ml.dmlc.xgboost4j.scala.spark.rapids import GpuDataReader
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from ml.dmlc.xgboost4j.scala.spark import XGBoostClassificationModel, XGBoostClassifier

In [0]:
from sklearn.datasets import fetch_openml
covtyp = fetch_openml(name="covertype", version=4mb)

In [12]:
covtyp.data.shape

(581012, 54)

In [14]:
np.unique(covtyp.target)

array(['1', '2', '3', '4', '5', '6', '7'], dtype=object)

In [0]:
# We are reading into pandas just for display
# We will use CUDA DF for analysis
cov_df = pd.DataFrame(data=np.c_[covtyp["data"], covtyp["target"]], 
                      columns=covtyp["feature_names"] + ["target"])

In [43]:
cov_df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,target
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5


In [44]:
# output- around 255 MBs
cov_df.memory_usage(index=True).sum()

255645408

In [45]:
cov_df.shape

(581012, 55)

In [46]:
cov_df.target.value_counts()

2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: target, dtype: int64

In [47]:
cov_df.dtypes[:5]

Elevation                           object
Aspect                              object
Slope                               object
Horizontal_Distance_To_Hydrology    object
Vertical_Distance_To_Hydrology      object
dtype: object

In [0]:
# All the columns are object type (string). To feed into any model we need to convert into 
# numeric data. We will do it using pd.to_numeric
# This method fails if column has non-numeric strings

In [0]:
for cols in cov_df.columns:
    cov_df[cols] = pd.to_numeric(cov_df[cols])

In [49]:
cov_df.dtypes[:5]

Elevation                           float64
Aspect                              float64
Slope                               float64
Horizontal_Distance_To_Hydrology    float64
Vertical_Distance_To_Hydrology      float64
dtype: object

In [50]:
np.unique(cov_df["target"])

array([1, 2, 3, 4, 5, 6, 7])

In [0]:
# These targets are from 1-7, but the normally we have indexing of target from 0.
# So we will minus 1 from all of them

In [0]:
cov_df["target"] = cov_df["target"] -1 

In [52]:
np.unique(cov_df["target"])

array([0, 1, 2, 3, 4, 5, 6])

In [0]:
# Parting into train and test
train_df = cov_df.sample(frac=0.8, random_state=10)
test_df = cov_df.drop(train_df.index)

In [54]:
train_df.shape

(464810, 55)

In [55]:
test_df.shape

(116202, 55)

In [56]:
# Here we are converting to parquet format and to store it and also using snappy compression to compress
train_df.to_parquet(fname = "covtype_train.parquet", compression="snappy", index=False)
test_df.to_parquet(fname="covtype_test.parquet", compression="snappy", index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: the 'fname'' keyword is deprecated, use 'path' instead
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: the 'fname'' keyword is deprecated, use 'path' instead
  


In [57]:
!ls

covtype_test.parquet	 spark-2.4.5-bin-hadoop2.7
covtype_train.parquet	 spark-2.4.5-bin-hadoop2.7.tgz
cudf-0.9.2-cuda10-1.jar  xgboost4j_2.x-1.0.0-Beta5.jar
sample_data		 xgboost4j-spark_2.x-1.0.0-Beta5.jar


In [0]:
import pyarrow.parquet as pq

In [0]:
pq.read_table("covtype_train.parquet")

In [62]:
train_data = GpuDataReader(spark).format("parquet").load("covtype_train.parquet")
test_data = GpuDataReader(spark).format("parquet").load("covtype_test.parquet")

TypeError: ignored